In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loan-data/de_duplicated_data.csv
/kaggle/input/home-credit-project/prev_approved_cash_loans.csv
/kaggle/input/home-credit-project/non_redundant_instal_payments.csv
/kaggle/input/home-credit-project/curr_cash_loans.csv
/kaggle/input/pos-cash-balance/POS_CASH_balance.csv
/kaggle/input/instal-data/non_redundant_instal.csv


In [2]:
from pathlib import Path
loan_data_dir = Path('/kaggle/input/loan-data')
output_dir = Path('/kaggle/working')

In [7]:
loan_df = pd.read_csv(loan_data_dir / 'de_duplicated_data.csv')
loan_df.drop('Unnamed: 0', axis=1, inplace=True)

In [6]:
loan_df.columns

Index(['Unnamed: 0', 'SK_ID_PREV', 'NUM_INSTALMENT_VERSION',
       'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT',
       'AMT_INSTALMENT', 'AMT_PAYMENT', 'TOTAL_REPAID_PER_INSTAL',
       'SK_ID_CURR', 'NAME_PRODUCT_TYPE', 'NAME_YIELD_GROUP', 'DAYS_FIRST_DUE',
       'DAYS_LAST_DUE', 'AMT_ANNUITY', 'AMT_CREDIT', 'AMT_GOODS_PRICE',
       'CNT_PAYMENT', 'INTEREST_RATE', 'TARGET', 'CODE_GENDER',
       'DAYS_ID_PUBLISH', 'CURR_AMT_CREDIT', 'CURR_AMT_ANNUITY',
       'NAME_CONTRACT_TYPE', 'AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE',
       'DAYS_EMPLOYED', 'FLAG_OWN_REALTY', 'NAME_HOUSING_TYPE',
       'NAME_EDUCATION_TYPE', 'LIVE_CITY_NOT_WORK_CITY',
       'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'FLAG_EMAIL',
       'FLAG_EMP_PHONE', 'FLAG_MOBIL', 'FLAG_CONT_MOBILE',
       'DEF_30_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE'],
      dtype='object')

In [11]:
loan_df.shape

(2839754, 39)

## Filter for previous cash loans with instalment histories which were at most 3 years-old.

In [13]:
at_most_3_yr_df = loan_df[np.abs(loan_df.DAYS_INSTALMENT) <= (365 * 3)]

pct_curr_id_preserved = (len(at_most_3_yr_df.SK_ID_CURR.unique()) / len(loan_df.SK_ID_CURR.unique())) * 100
pct_prev_id_preserved = (len(at_most_3_yr_df.SK_ID_PREV.unique()) / len(loan_df.SK_ID_PREV.unique())) * 100
print(f'PCT SK_ID_CURR: {pct_curr_id_preserved}')
print(f'PCT SK_ID_PREV: {pct_prev_id_preserved}')

PCT SK_ID_CURR: 95.80176820232764
PCT SK_ID_PREV: 85.96217391304349


In [15]:
test_df = at_most_3_yr_df[['SK_ID_CURR', 'TARGET']].copy().drop_duplicates()
test_df.groupby('TARGET').count() #There's 9439 Defaulters for now.

,SK_ID_CURR
TARGET,
0,105640
1,9439


## Compute Feature 1:
## Weighted Average Interest for Previous Loans with Instalment Histories which were at most 3 years old.

In [17]:
cols_to_keep = ['SK_ID_CURR', 'SK_ID_PREV', 'AMT_CREDIT', 'INTEREST_RATE']
interest_df = at_most_3_yr_df.copy()[cols_to_keep].drop_duplicates()

In [26]:
interest_df['TOTAL_AMT_CREDIT'] = interest_df \
                                    .groupby('SK_ID_CURR')['AMT_CREDIT'] \
                                    .transform(np.sum)

interest_df['WEIGHT'] = interest_df['AMT_CREDIT'] / interest_df['TOTAL_AMT_CREDIT']
interest_df['INTEREST_RATE_ADJ'] = interest_df['WEIGHT']  * interest_df['INTEREST_RATE']

interest_df['WEIGHTED_AVR_INTEREST_RATE'] = interest_df \
                                                .groupby('SK_ID_CURR')['INTEREST_RATE_ADJ'] \
                                                .transform(np.sum)

interest_feature_df = interest_df[['SK_ID_CURR', 'WEIGHTED_AVR_INTEREST_RATE']] \
                        .copy().drop_duplicates()
interest_feature_df.head()

,SK_ID_CURR,WEIGHTED_AVR_INTEREST_RATE
0,198678,45.090000
12,277601,33.370296
23,198771,40.000000
33,217553,67.426297
48,330786,35.526539


In [28]:
len(interest_feature_df.SK_ID_CURR.unique()) == len(at_most_3_yr_df.SK_ID_CURR.unique()) #sanity check

True

## Compute the following metrics:
1. Sum of Days Past Due for instalments which were at most 5 years old.
2. Sum of Money Owed for instalments which were at most 5 years old.
3. Number of Late Payment streaks for instalments which were at most 5 years old.
4. Number of Consecutive Partial Payment streaks for instalments which were at most 5 years old.
5. Compute loan interest rates.
Note: all these features  are only relevant for curr loan ids in five_yr_instal df

### Total Days Past Due for instalments from previous loans
**PREV_INSTALMENT_PAID_DAY**: The actual day which a borrower paid his or her instalment. <br>
For eg: if **PREV_INSTALMENT_PAID_DAY** is -10, a borrower repaid his or her instalment 10 days prior to current loan application.

**PREV_INSTALMENT_DUE_DAY**: The deadline to pay an instalment. <br>
For eg: if **PREV_INSTALMENT_DUE_DAY** is -20, a borrower needs to repay his or her instalment 20 days prior to current loan application. <br>

In our example, **PREV_INSTALMENT_PAID_DAY** occurred after **PREV_INSTALMENT_DUE_DAY**, the borrower was late in his or her repayment.


In [ ]:
five_yr_instal_BU = five_yr_instal.copy()
five_yr_instal_BU['DAYS_PAST_DUE'] = five_yr_instal_BU['PREV_INSTALMENT_PAID_DAY'] \
                                        - five_yr_instal_BU['PREV_INSTALMENT_DUE_DAY']

sum_dpd_by_previous_loans = five_yr_instal_BU.groupby(
    ['CURR_CUSTOMER_ID', 'PREV_LOAN_ID'])['DAYS_PAST_DUE'].sum().reset_index()

In [ ]:
sum_dpd_by_curr_customer = sum_dpd_by_previous_loans.groupby(
                                ['CURR_CUSTOMER_ID'])['DAYS_PAST_DUE'].sum().reset_index()

#ensure all curr customer IDs are preserved.
set(sum_dpd_by_curr_customer.CURR_CUSTOMER_ID.unique()) == set(sense_check.CURR_CUSTOMER_ID.unique())

In [ ]:
#rename and save
new_names = {'DAYS_PAST_DUE': 'TOTAL_PAST_DPD'} #Where DPD == DAYS_PAST_DUE
sum_dpd_by_curr_customer.rename(
    new_names, axis=1, inplace=True)

sum_dpd_by_curr_customer.to_csv(
    output_dir / 'total_prev_dpd_by_customer.csv', index=False)

### Sum of Money Owed
In installments dataset, each instalment payment can be split into more than 1 rows. <br>
To compute total amount of money owed, I do the following:
1. Compute **Total Repaid Per Instalment** via a groupby, store this as a table.
2. Join the table from step 1 with table containing previously approved cash loans.
3. To determine amount of money owed to Home Credit, compute **PREV_MONTHLY_INSTALMENT_AMT - total payment per instalment**. Each Instalment and Amount Annuity is monthly.

In [ ]:
five_yr_instal_BU = five_yr_instal.copy()

total_repaid_per_instal = five_yr_instal_BU.groupby(
    ['PREV_LOAN_ID', 'NUM_INSTALMENT_NUMBER', 'PREV_MONTHLY_INSTALMENT_AMT'])['PREV_MONTHLY_REPAID_AMT'].sum().reset_index()

total_repaid_per_instal.rename(
    {'PREV_MONTHLY_REPAID_AMT': 'TOTAL_REPAID_PER_INSTAL'}, 
axis=1, inplace=True)

In [ ]:
total_repaid_per_instal['PREV_MONTHLY_INSTALMENT_AMT'] = round(
    total_repaid_per_instal['PREV_MONTHLY_INSTALMENT_AMT'])

total_repaid_per_instal['TOTAL_REPAID_PER_INSTAL'] = round(
    total_repaid_per_instal['TOTAL_REPAID_PER_INSTAL'])

In [ ]:
total_repaid_per_instal['MONEY_OWED_PER_INSTAL'] = (
        total_repaid_per_instal['PREV_MONTHLY_INSTALMENT_AMT'] - 
            total_repaid_per_instal['TOTAL_REPAID_PER_INSTAL'])

In [ ]:
total_repaid_per_prev_loan = total_repaid_per_instal.groupby(
    ['CURR_CUSTOMER_ID', 'PREV_LOAN_ID'])['MONEY_OWED_PER_INSTAL'].sum().reset_index()

total_repaid_per_curr_client = total_repaid_per_prev_loan.groupby(
    ['CURR_CUSTOMER_ID'])['MONEY_OWED_PER_INSTAL'].sum().reset_index()

### Compute Frequency of Late Payments
1. A payment can be 1 late payment or a streak of consecutive late payments. See [source](https://joshdevlin.com/blog/calculate-streaks-in-pandas/) and see **compute_total_late_payment** function.
2. Create a new column called is_late.
3. Based on our formula for Days Past Due, is_late if True is Days Past Due is Positive.

In [ ]:
def compute_total_late_payment(instal_info:pd.DataFrame) -> pd.DataFrame:
    """
        Computes number of late payments (can be just 1 late payment or a streak of late payments)
        for just 1 previous loan as denoted by SK_ID_PREV.
        
        Parameters:
        -----------
            instal_info (pd.DataFrame): 
                A dataframe of instalment history for a current borrower for 1 previous loan.
                instal_info must contain the following information:
                    1. SK_ID_CURR.
                    2. SK_ID_PREV.
                    3. IS_LATE.
        
        Returns:
       ----------
       A DataFrame containing 3 pieces of information:
       1. Cumulative count of streaks of late payments for a previous loan.
       2. SK_ID_CURR.
       3. SK_ID_PREV.
       
    """
    instal_info['grouper'] = (instal_info.IS_LATE != instal_info.IS_LATE.shift()).cumsum()
    subset = instal_info[['grouper', 'IS_LATE']]
    
    subset['streaks_count'] = subset.groupby('grouper').cumsum()
    subset['start_of_streak'] = (subset['streaks_count'] == 1) | (subset['streaks_count'] == 0)
    subset['end_of_streak'] = (subset['streaks_count'].shift(-1) == 1) | (subset['streaks_count'].shift(-1) == 0)| (subset['streaks_count'].shift(-1).isnull())
    
    total_late_payments = subset[subset['end_of_streak']]['streaks_count'].sum()
    
    sk_id_curr = instal_info['SK_ID_CURR'].unique()[0]
    sk_id_prev = instal_info['SK_ID_PREV'].unique()[0]
    
    new_data = {'SK_ID_CURR': [sk_id_curr], 
                'SK_ID_PREV': [sk_id_prev], 
                'total_late_payment': [total_late_payments]}
    
    return pd.DataFrame.from_dict(new_data)

In [ ]:
cols_to_show = ['SK_ID_CURR', 'SK_ID_PREV', 'DAYS_PAST_DUE']
instal_copy = three_yr_instal_1.copy()[cols_to_show]
instal_copy['IS_LATE'] = instal_copy['DAYS_PAST_DUE'] > 0
instal_copy.head()

In [ ]:
"""

total_late_payments = []

count = 0
for sk_id_prev in instal_copy.SK_ID_PREV.unique():
    prev_loan_df = instal_copy[instal_copy.SK_ID_PREV == sk_id_prev]
    late_payment_df = compute_total_late_payment(prev_loan_df[['SK_ID_PREV', 'SK_ID_CURR', 'IS_LATE']])
    total_late_payments.append(late_payment_df)
    count += 1
    if count % 10_000 == 0:
        print(count)

late_payments_df = pd.concat(total_late_payments)
late_payments_df.head()
"""

In [ ]:
late_payments_df.rename({'total_late_payment': 'total_late_payments_freq'}, axis=1, inplace=True)
late_payments_df.to_csv(output_dir / 'late_instal_payments_count.csv', index=False)
late_payments_df.head()

### Compute Frequency of Partial Repayment of Instalments
1. Computation is similar to Frequency of Late Payments.
2. I only considered amount of money owed which exceed 100 dollars.

In [ ]:
def compute_total_partial_payment(instal_info:pd.DataFrame) -> pd.DataFrame:
    """
        Computes number of partial instalment payments (can be just 1 partial payment or a streak of partial payments)
        for just 1 previous loan as denoted by SK_ID_PREV.
        
        Parameters:
        -----------
            instal_info (pd.DataFrame): 
                A dataframe of instalment history for a current borrower for 1 previous loan.
                instal_info must contain the following information:
                    1. SK_ID_CURR.
                    2. SK_ID_PREV.
                    3. IS_PARTIAL_REPAYMENT.
        
        Returns:
       ----------
       A DataFrame containing 3 pieces of information:
       1. Cumulative count of streaks of partial payments for a previous loan.
       2. SK_ID_CURR.
       3. SK_ID_PREV.
       
    """
    instal_info['grouper'] = (instal_info.IS_PARTIAL_REPAYMENT != instal_info.IS_PARTIAL_REPAYMENT.shift()).cumsum()
    subset = instal_info[['grouper', 'IS_PARTIAL_REPAYMENT']]
    
    subset['streaks_count'] = subset.groupby('grouper').cumsum()
    subset['start_of_streak'] = (subset['streaks_count'] == 1) | (subset['streaks_count'] == 0)
    subset['end_of_streak'] = (subset['streaks_count'].shift(-1) == 1) | (subset['streaks_count'].shift(-1) == 0)| (subset['streaks_count'].shift(-1).isnull())
    
    total_late_payments = subset[subset['end_of_streak']]['streaks_count'].sum()
    
    sk_id_curr = instal_info['SK_ID_CURR'].unique()[0]
    sk_id_prev = instal_info['SK_ID_PREV'].unique()[0]
    
    new_data = {'SK_ID_CURR': [sk_id_curr], 
                'SK_ID_PREV': [sk_id_prev], 
                'total_partial_payment': [total_late_payments]}
    
    return pd.DataFrame.from_dict(new_data)

In [ ]:
cols_to_keep = ['SK_ID_PREV', 'SK_ID_CURR', 'NUM_INSTALMENT_NUMBER', 'MONEY_OWED']
partial_repay = instal_and_prev_apps.copy()[cols_to_keep]
partial_repay['IS_PARTIAL_REPAYMENT'] = partial_repay.MONEY_OWED > 100

In [ ]:
total_partial_payments = []

count = 0
for sk_id_prev in partial_repay.SK_ID_PREV.unique():
    repay_df = partial_repay[partial_repay.SK_ID_PREV == sk_id_prev]
    partial_payment_df = compute_total_partial_payment(repay_df[['SK_ID_PREV', 'SK_ID_CURR', 'IS_PARTIAL_REPAYMENT']])
    total_partial_payments.append(partial_payment_df)
    count += 1
    if count % 10_000 == 0:
        print(count)

partial_payments_df = pd.concat(total_partial_payments)
partial_payments_df.head()

In [ ]:
partial_payments_df.to_csv(output_dir/'partial_payments_count.csv', index=False)